In [ ]:
#include <iostream>
#include <thread>
#include <vector>
#include <future>
#include <mutex>
#include<algorithm>

# Using mutex to protect data

In [ ]:
class Vehicle
{
public:
    Vehicle(int id) : _id(id) {}

private:
    int _id;
};

In [ ]:
class WaitingVehicles
{
public:
    WaitingVehicles() {}

    // getters / setters
    void printSize()
    {
        _mutex.lock();
        std::cout << "#vehicles = " << _vehicles.size() << std::endl;
        _mutex.unlock();
    }

    // typical behaviour methods
    void pushBack(Vehicle &&v)
    {
        _mutex.lock();
        _vehicles.emplace_back(std::move(v)); // data race would cause an exception
        _mutex.unlock();
    }

private:
    std::vector<Vehicle> _vehicles; // list of all vehicles waiting to enter this intersection
    std::mutex _mutex;
};

# Using timed_mutex

In the following, a short overview of the different available mutex types is given:

- `mutex`: provides the core functions lock() and unlock() and the non-blocking try_lock() method that returns if the mutex is not available.

- `recursive_mutex`: allows multiple acquisitions of the mutex from the same thread.

- `timed_mutex`: similar to mutex, but it comes with two more methods try_lock_for() and try_lock_until() that try to acquire the mutex for a period of time or until a moment in time is reached.

- `recursive_timed_mutex`: is a combination of timed_mutex and recursive_mutex.

# Deadlock 1

Using mutexes can significantly reduce the risk of data races as seen in the example above. But imagine what would happen if an exception was thrown while executing code in the critical section, i.e. between lock and unlock. In such a case, the mutex would remain locked indefinitely and no other thread could unlock it - the program would most likely freeze.

In [ ]:
void divideByNumber(double num, double denom)
{
    try
    {
        // divide num by denom but throw an exception if division by zero is attempted
        if (denom != 0) 
        {
            result = num / denom;
            std::this_thread::sleep_for(std::chrono::milliseconds(1)); 
            printResult(denom);
        }
        else
        {
            throw std::invalid_argument("Exception from thread: Division by zero!");
        }
    }
    catch (const std::invalid_argument &e)
    {
        // notify the user about the exception and return
        std::cout << e.what() << std::endl;
        return; 
    }
}

# Deadlock 2

A second type of deadlock is a state in which two or more threads are blocked because each thread waits for the resource of the other thread to be released before releasing its resource. The result of the deadlock is a complete standstill.

In [ ]:
std::mutex mutex1, mutex2;

In [ ]:
void ThreadA()
{
    // Creates deadlock problem
    mutex2.lock();
    std::cout << "Thread A" << std::endl;
    mutex1.lock();
    mutex2.unlock();
    mutex1.unlock();
}

In [ ]:
void ThreadB()
{
    // Creates deadlock problem
    mutex1.lock();
    std::cout << "Thread B" << std::endl;
    mutex2.lock();
    mutex1.unlock();
    mutex2.unlock();
}

# Avoiding deadlocks with `std::lock()`

In [ ]:
void ThreadA()
{
    // Ensure that locks are always executed in the same order
    // Locks all the objects passed as arguments, blocking the calling thread if necessary.
    std::lock(mutex1, mutex2);
    // std::adopt_lock : assume the calling thread already has ownership of the mutex
    // because lock has been done in the previous step
    std::lock_guard<std::mutex> lock2(mutex2, std::adopt_lock);
    std::cout << "Thread A" << std::endl;
    std::lock_guard<std::mutex> lock1(mutex1, std::adopt_lock);
    std::cout << "Thread A" << std::endl;
}

In [ ]:
void ThreadB()
{
    std::lock(mutex1, mutex2);
    std::lock_guard<std::mutex> lock1(mutex1, std::adopt_lock);
    std::cout << "Thread B" << std::endl;
    std::lock_guard<std::mutex> lock2(mutex2, std::adopt_lock);
    std::cout << "Thread B" << std::endl;
}

In [ ]:
void ExecuteThreads()
{
    std::thread t1( ThreadA );
    std::thread t2( ThreadB );
 
    t1.join();
    t2.join();
 
    std::cout << "Finished" << std::endl;
}

In [ ]:
ExecuteThreads();

# lock_guard Vs. unique_lock

- `std::lock_guard` :
  - `std::lock_guard guard1(mutex);` : the constructor of guard1 locks the mutex. At the end of guard1’s life, the destructor unlocks the mutex. There is no other possibility. In fact, the std::lock_guard class doesn’t have any other member function.

- `std::unique_lock` :
  - `std::unique_lock guard2(mutex);` : The constructor of guard2 also locks the mutex and the destructor of guard2 also unlocks the mutex. But the std::unique_lock has additional functionalities.
     1. The programmer is able to unlock the mutex with the help of the guard object `guard2.unlock();`
     1. The programmer can also lock it again `guard2.lock();`
     
  - `std::condition_variable::wait()`: 
    - requires std::unique_lock as an input.
    - unlocks the mutex and waits for the `std::condition_variable.notify_one()` member function call. Then, `wait(...)` reacquires the lock and proceeds.

# Condition Variables

Purposes:
1. Notify other threads:
   1. `notify_one()`
   1. `notify_all()`
1. Waiting for some conditions

Producer/Consumer:
1. Consumer:
   1. use `std::unique_lock` to pass it to the `std::condition_variable` because `std::condition_variable::wait()` will use it to unloack the `shared_mutex` before start waiting.
   1. `std::condition_variable::wait()` unloack the `shared_mutex` and wait to be notified in order to continue.
   1. When producer call `notify()`,  The `std::condition_variable::wait()` will lock `shared_mutex` again to continue execution.
1. Producer:
   1. accuire `shared_mutex` using `std::lock_guard` in order not to conflict with `step3 consumer`.
   2. call `notify()`

In [1]:
#include <iostream>
#include <thread>
#include <vector>
#include <future>
#include <mutex>

In [2]:
std::condition_variable cv;
std::mutex m;
int balance = 0;

In [3]:
void Producer()
{
    std::lock_guard<std::mutex> lg(m);
    balance += 100;
    std::cout << "Amount added" << std::endl;
    cv.notify_one();
}

In [4]:
void Consumer()
{
    std::unique_lock<std::mutex> ul(m);
    cv.wait(ul, []{return balance != 0;});
    balance -= 50;
    std::cout << "Amount deducted" << std::endl;
}

In [5]:
std::thread t1(Consumer);
std::thread t2(Producer);
t1.join();
t2.join();

Amount added
Amount deducted
